*O objetivo desse notebook é ler uma base de dados de vendas em uma loja, analisar e enviar o relatório por email.*<br>
Os passos do que preciso fazer são:
- Saber o faturamento por loja
- Calcular a quantidade de produtos vendidos por loja
- Saber qual o ticket médio (calcular o faturamento e dividir pela quantidade de produtos vendidos) por produto em cada loja
- Enviar um email com o relatório


**Importar a base de dados**

In [1]:
import pandas as pd

In [2]:
df_vendas = pd.read_excel('Vendas.xlsx')

**Visualizar as 5 primeiras linhas da tabela (dataframe)**

In [3]:
df_vendas.head()

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,Iguatemi Esplanada,Sapato Estampa,1,358,358
1,1,2019-01-01,Iguatemi Esplanada,Camiseta,2,180,360
2,1,2019-01-01,Iguatemi Esplanada,Sapato Xadrez,1,368,368
3,2,2019-01-02,Norte Shopping,Relógio,3,200,600
4,2,2019-01-02,Norte Shopping,Chinelo Liso,1,71,71


**Agrupar as lojas pra saber o faturamento**

Vou filtrar as colunas e trazer só a coluna 'ID Loja' e o 'Valor final', agrupar as lojas usando o `groupby()` e somar as vendas usando `sum()` e colocar dentro do dataframe 'df_faturamento'

In [4]:
df_vendas.columns

Index(['Código Venda', 'Data', 'ID Loja', 'Produto', 'Quantidade',
       'Valor Unitário', 'Valor Final'],
      dtype='object')

In [5]:
df_faturamento = df_vendas[['ID Loja','Valor Final']].groupby('ID Loja').sum()
df_faturamento.head()

,Valor Final
ID Loja,
Bourbon Shopping SP,1726110
Center Shopping Uberlândia,1668921
Iguatemi Campinas,1762419
Iguatemi Esplanada,1699681
Norte Shopping,1711968


**Calcular a quantidade de produtos vendidos por loja**<br>
Usei o `sort_values()` para ordenar os resultados. Disse no parametro *by* por qual coluna quero ordenar e no *ascending* qual ordem (crescente ou decrescente).

In [6]:
df_qtdProdutos = df_vendas[['ID Loja','Quantidade']].groupby('ID Loja').sum().sort_values(by='Quantidade', ascending = False)
df_qtdProdutos.head()

,Quantidade
ID Loja,
Shopping Vila Velha,9224
Palladium Shopping Curitiba,9091
Norte Shopping,9014
Shopping Center Leste Aricanduva,8938
Rio Mar Shopping Fortaleza,8937


**Calcular o ticket médio por produto em cada loja**
Coloquei o cálculo entre parênteses e chamei o método `to_frame()` para o resultado ser apresentado como um dataframe (tabela)

In [7]:
df_ticket_medio = (df_faturamento['Valor Final'] / df_qtdProdutos['Quantidade']).to_frame()
df_ticket_medio.head()

,0
ID Loja,
Bourbon Shopping SP,194.754598
Center Shopping Uberlândia,193.453228
Iguatemi Campinas,197.248909
Iguatemi Esplanada,198.098019
Norte Shopping,189.923231


Quando é visualizado o dataframe, ele aponta o nome da coluna onde está os valores como '0'. Vou renomear a coluna de nome '0' para o nome 'Ticket Médio'.

In [27]:
df_ticket_medio = df_ticket_medio.rename(columns={0:'Ticket Médio'})
df_ticket_medio.head()

,Ticket Médio
ID Loja,
Bourbon Shopping SP,194.754598
Center Shopping Uberlândia,193.453228
Iguatemi Campinas,197.248909
Iguatemi Esplanada,198.098019
Norte Shopping,189.923231


**Integrar o python com um gerenciador de email**<br>
Para dá certo, é preciso configurar o outlook no computador

In [9]:
import smtplib
import email.message

In [29]:
def enviar_email(): 
    #Qual o conteudo do email. O .t0_html() é pra o dataframe ficar uma tabela html. Usei o formatters para incluir um 'R$' na 
    #frente do valor monetário e usei {:,.2f}pra formatar o numero, sendo a virgula para separar em milhar e o .2f separar em 
    #duas casas decimais
    
    corpo_email = f'''
<p>Oi Linda, </p>
<p>pega esse relatorio ai na moral</p>

<p>Faturamento</p>
{df_faturamento.to_html(formatters={'Valor Final':'R${:,.2f}'.format})}

<p>Quantidade Vendida</p>
{df_qtdProdutos.to_html()}

<p>Ticket Médio dos produtos em cada loja</p>
{df_ticket_medio.to_html(formatters={'Ticket Médio':'R${:,.2f}'.format})}

<p>grands beijinhos</p>

'''

    msg = email.message.Message()
    #Qual o assunto do email
    msg['Subject'] = 'Relatorio de vendas por loja'
    #Qual o endereço de email de quem envia
    msg['From'] = 'email do remetente'
    #Qual o endereço de email para quem vai enviar
    msg['To'] = 'email do destinatario '
    #senha do email do remetente
    password = 'senha do remetente'
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(corpo_email )

    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    #credenciais de login para enviar o email
    s.login(msg['From'], password)
    s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    print('Email enviado')



In [30]:
enviar_email()

Email enviado


**OBS**: Se ao rodas `enviar_email()` aparecer o erro um erro, ou sua senha está errada ou seu email está com a camada de segurança do google. Você precisa habilitar o acesso a app menos seguro, para isso basta ir em **gerencias sua conta google > segurança > acesso a app menos seguro**. Não esqueça de habilitar novamente depois ou sua conta ficará vulnerável.